In [1]:
%run ./../../utils/commonImports.py
%run ./../../utils/tradingImports.py
%matplotlib inline

# Load Data

In [2]:
db = 'D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data\\USDT_BTC.db'
data = load_trading_data(db, 'chart_data', from_date='2014-01-01', period='5min')

In [3]:
data.shape

(381242, 10)

# Experiment

Celý experiment spočívá v olabelovaní dat následujícím způsobem. Pokud daný bod při nákupu vydělá více než 1% tak True.
Snaha bude naučit klasifikátor tak, aby měl co nejlepší precision a zároveň provedl co nejvíce obchodů.

# Calculation

In [4]:
import time

def _max_row(day_df, col):
    i = day_df[col].idxmax()
    max_row = day_df.iloc[i, :]
    return max_row


def _min_row(day_df, col):
    i = day_df[col].idxmin()
    min_row = day_df.iloc[i, :]
    return min_row


def _max(day_df):
    close_max_row = _max_row(day_df, 'close')
    open_max_row = _max_row(day_df, 'open')
    return (close_max_row, 'close') if close_max_row['close'] > open_max_row['open'] else (open_max_row, 'open')


def _fulfilled_row(row, day_df, col, target):
    day_df['R'] = _return(row['close'], day_df[col])
    fulfilled = day_df.query(f"R >= {target}")
    if fulfilled.shape[0] > 0:
        return fulfilled.iloc[0, :]
    else:
        return day_df.iloc[-1, :]


def fulfilled(row, day_df, target):
    close_row = _fulfilled_row(row, day_df, 'close', target)
    open_row = _fulfilled_row(row, day_df, 'open', target)
    return (close_row, 'close') if close_row['close'] > open_row['open'] else (open_row, 'open')


def _return(v1, v2):
    return (v2 / v1) - 1

In [5]:
maxs = []
till_maxs = []
mins = []
df = data.copy()
DAY = 288
TARGET = 0.01

start_time = time.time()
for i in range(df.shape[0] - 2):
    if i % 1000 == 0:
        print(i, '%.3f s' % (time.time() - start_time))
        start_time = time.time()
    day_i = i + DAY if i + DAY < df.shape[0] else df.shape[0]
    row = df.iloc[i, :]
    day_df = reset_index_hard(df.iloc[i+1:day_i, :]).reset_index()
    
    fullfiled_row, fullfiled_col = fulfilled(row, day_df, TARGET)
    
    max_index = fullfiled_row['index'] if fullfiled_row['index'] > 0 else 1
    day_df = day_df.iloc[:max_index, :]
    min_row = _min_row(day_df, 'low')
    
    maxs.append(_return(row['close'], fullfiled_row[fullfiled_col]))
    till_maxs.append(fullfiled_row['index'])
    mins.append(_return(row['close'], min_row['low']))

0 0.000 s
1000 12.991 s
2000 13.936 s
3000 13.929 s
4000 11.527 s
5000 10.583 s
6000 14.829 s
7000 14.442 s
8000 11.011 s
9000 11.280 s
10000 15.309 s
11000 15.422 s
12000 14.172 s
13000 11.372 s
14000 11.100 s
15000 13.151 s
16000 10.989 s
17000 10.812 s
18000 11.332 s
19000 12.336 s
20000 15.335 s
21000 13.024 s
22000 12.906 s
23000 13.511 s
24000 12.716 s
25000 12.737 s
26000 12.935 s
27000 13.032 s
28000 12.615 s
29000 13.061 s
30000 12.682 s
31000 12.787 s
32000 12.715 s
33000 12.920 s
34000 12.699 s
35000 12.764 s
36000 12.707 s
37000 12.685 s
38000 12.791 s
39000 12.680 s
40000 12.590 s
41000 14.273 s
42000 10.938 s
43000 10.329 s
44000 10.351 s
45000 10.342 s
46000 10.308 s
47000 10.416 s
48000 10.342 s
49000 10.381 s
50000 10.389 s
51000 10.408 s
52000 10.348 s
53000 10.299 s
54000 10.354 s
55000 10.759 s
56000 10.349 s
57000 10.310 s
58000 10.294 s
59000 10.284 s
60000 12.015 s
61000 12.565 s
62000 10.980 s
63000 10.833 s
64000 12.214 s
65000 11.585 s
66000 10.398 s
67000 11.

In [6]:
maxs += [-1, -1]
till_maxs += [-1, -1]
mins += [-1, -1]

In [7]:
df['maxReturn'] = maxs
df['tillMax'] = till_maxs
df['minReturn'] = mins

In [8]:
df

,timestamp,period,open,high,low,close,volume,quoteVolume,weightedAverage,date,maxReturn,tillMax,minReturn
0,1424373000,5min,225.0000,225.0000,225.0000,225.0000,1.0000,0.0044,225.0000,2015-02-19 19:10:00,0.0667,14,0.0000
1,1424373300,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:15:00,0.0667,13,0.0000
2,1424373600,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:20:00,0.0667,12,0.0000
3,1424373900,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:25:00,0.0667,11,0.0000
4,1424374200,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:30:00,0.0667,10,0.0000
5,1424374500,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:35:00,0.0667,9,0.0000
6,1424374800,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:40:00,0.0667,8,0.0000
7,1424375100,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:45:00,0.0667,7,0.0000
8,1424375400,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:50:00,0.0667,6,0.0000
9,1424375700,5min,225.0000,225.0000,225.0000,225.0000,0.0000,0.0000,225.0000,2015-02-19 19:55:00,0.0667,5,0.0000


In [9]:
write_tsv(df, 'D:\\Dropbox\\My work\\krypl-project\\analysis\\return-01-within-day.tsv')